In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
stud = pd.read_csv("/kaggle/input/anlise-preditiva-de-evaso-acadmica/data.csv",sep=";")
prod = pd.read_csv("/kaggle/input/bike-store-sample-database/products.csv")
prod.head()

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,Trek 820 - 2016,9,6,2016,379.99
1,2,Ritchey Timberwolf Frameset - 2016,5,6,2016,749.99
2,3,Surly Wednesday Frameset - 2016,8,6,2016,999.99
3,4,Trek Fuel EX 8 29 - 2016,9,6,2016,2899.99
4,5,Heller Shagamaw Frame - 2016,3,6,2016,1320.99


# 2. Funções do Pandas

In [2]:
# Qual a proporção de Dropouts
stud["Target"].value_counts(1)

Target
Graduate    0.499322
Dropout     0.321203
Enrolled    0.179476
Name: proportion, dtype: float64

In [3]:
# Qual a média de inflação para cada tipo de target
stud.groupby("Target")["Inflation rate"].mean()

Target
Dropout     1.283955
Enrolled    1.211713
Graduate    1.197918
Name: Inflation rate, dtype: float64

****Como podemos ver, temos uma taxa de evasão (dropout) bem alta, cerca de 32% dos alunos deixam a graduação. Quando olhamos a inflação do momento em que o aluno se graduou (Graduate) ou deixou a universidade (Dropout), temos uma diferença de 0.07. Como a taxa está em %, estamos falando de 0.07%, o que não seria tão alto assim. Em outras palavras, a inflação parece não ter correlação com a decisão do aluno de evadir o curso.

# 2.1. Pivotamento de Tabelas (Pivot Tables)

In [4]:
stud.pivot_table(index='Target', columns='Marital status', values='Inflation rate', aggfunc='mean')

Marital status,1,2,3,4,5,6
Target,,,,,,
Dropout,1.282601,1.300559,1.40,1.269048,1.354545,0.875
Enrolled,1.220694,1.003846,2.55,1.593750,-0.333333,1.400
Graduate,1.193499,1.216216,-0.80,1.163636,2.181818,-0.300


****Agora utilizando uma tabela dinâmica, utilizando o Target e o Marital Status, podemos tirar a média de evasão por status matrimônial, isso é impresisonante pois nos deixa analisar de formas diferentes os dados.

In [5]:
stud.pivot_table(index='Target',
                 columns = ['Daytime/evening attendance\t', 'Marital status'],
                 values = 'Previous qualification (grade)',
                 aggfunc = 'mean'
                )

Daytime/evening attendance	           0                                        \
Marital status                        1           2      3           4      5   
Target                                                                          
Dropout                      128.471774  130.476471    NaN  127.733333  120.0   
Enrolled                     128.863415  133.296000    NaN  135.400000    NaN   
Graduate                     132.313333  131.133784  120.0  138.388235  120.0   

Daytime/evening attendance	                  1                                 \
Marital status                   6           1           2      3           4   
Target                                                                          
Dropout                      122.5  131.537736  129.937838  135.0  130.753333   
Enrolled                     133.1  131.172607  129.177778  154.0  137.412500   
Graduate                     133.1  134.495393  130.505405    NaN  126.593750   

Daytime/evening attendance	                   
Marital status                      5      6  
Target                                        
Dropout                      143.5500  133.1  
Enrolled                     128.7000    NaN  
Graduate                     132.1375    NaN

**Veja que agora a gente além de quebrar por tipo de target e por matrimônio, ainda olhamos para o horário que a pessoa atendia às aulas. Ou seja, você conseguirá diferenciar a nota da última qualificação (previous qualification (grade)) do solteiro que evadiu e estuda à noite do solteiro que evadiu e estuda de dia (além de todos os outros status, claro). Imagine o quanto que isso será poderoso nas suas análises!**

# 2.2 Formatação Condicional

In [6]:
# Criando um DataFrame simples
df = pd.DataFrame({
    'A': [9, -7, 5],
    'B': [-1, 3, -4]
})

df

,A,B
0,9,-1
1,-7,3
2,5,-4


In [7]:
# Definindo uma função para aplicar a coloração
def color_positive_negative(val):
    color = '#008000' if val > 0 else '#FF0000'
    return 'color: %s'% color

# Aplicando a coloração ao DataFrame
styled_df = df.style.map(color_positive_negative)
display(styled_df)

,A,B
0,9,-1
1,-7,3
2,5,-4


In [8]:
# Defube a função que destaca máximo e mínimo
def highlight_max_min(s):
    is_max = s == s.max()
    is_min = s == s.min()
    return ['color: green' if v_max else 'color: red' if v_min else '' for v_max, v_min in zip (is_max, is_min)]

# Aplica formatação condicional
styled_df = df.style.apply(highlight_max_min)
display(styled_df)

,A,B
0,9,-1
1,-7,3
2,5,-4


**Agora vamos utilizar formatação condicional em uma tabela dinâmica**

In [9]:
pivot_df = stud.pivot_table(index = 'Marital status',
                    columns = ['Target'],
                    values = 'Previous qualification (grade)',
                    aggfunc = 'mean'
                    )
pivot_df

Target,Dropout,Enrolled,Graduate
Marital status,,,
1,131.216639,131.041111,134.381687
2,130.142458,131.157692,130.819595
3,135.000000,154.000000,120.000000
4,129.890476,136.406250,132.669697
5,141.409091,128.700000,128.827273
6,127.800000,133.100000,133.100000


In [10]:
def highlight_max_min(data):
    styles = data.copy()
    for col in data.columns:
        max_val = data[col].max()
        min_val = data[col].min()
        styles[col] = ['background-color: lightgreen' if v == max_val else 'background-color: yellow'
                      if v== min_val else '' for v in data[col]
                      ]
    return styles

styled_df = pivot_df.style.apply(highlight_max_min, axis = None)
display(styled_df)

Target,Dropout,Enrolled,Graduate
Marital status,,,
1,131.216639,131.041111,134.381687
2,130.142458,131.157692,130.819595
3,135.000000,154.000000,120.000000
4,129.890476,136.406250,132.669697
5,141.409091,128.700000,128.827273
6,127.800000,133.100000,133.100000


# 2.3 Usando a função agg no Pandas

Nós já vimos como gerar algumas estatísticas para as colunas, de forma isolada, e para agrupamentos de categorias no Pandas. Entretanto, ainda não vimos como poderíamos montar uma tabela com diversas estatísticas descritivas para diferentes colunas. A função agg é usada para aplicar estes tipos de operações de agregação a colunas do DataFrame!

In [11]:
# Filtrar apenas colunas numéricas
numeric_cols = stud.select_dtypes(include=[np.number])

# Calcular a média das colunas numéricas
mean_values = numeric_cols.agg("mean", axis=0)
print(mean_values)


Marital status                                       1.178571
Application mode                                    18.669078
Application order                                    1.727848
Course                                            8856.642631
Daytime/evening attendance\t                         0.890823
Previous qualification                               4.577758
Previous qualification (grade)                     132.613314
Nacionality                                          1.873192
Mother's qualification                              19.561935
Father's qualification                              22.275316
Mother's occupation                                 10.960895
Father's occupation                                 11.032324
Admission grade                                    126.978119
Displaced                                            0.548373
Educational special needs                            0.011528
Debtor                                               0.113698
Tuition 

In [12]:

numeric_cols.agg(['sum', 'mean', 'min'], axis=0)


,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
sum,5214.000000,82592.000000,7644.000000,3.918179e+07,3941.000000,20252.000000,586681.300000,8287.000000,86542.000000,98546.000000,...,609.000000,2397.000000,27571.000000,35672.000000,19624.000000,45258.430117,665.000000,51168.600000,5432.800000,8.710000
mean,1.178571,18.669078,1.727848,8.856643e+03,0.890823,4.577758,132.613314,1.873192,19.561935,22.275316,...,0.137658,0.541817,6.232143,8.063291,4.435805,10.230206,0.150316,11.566139,1.228029,0.001969
min,1.000000,1.000000,0.000000,3.300000e+01,0.000000,1.000000,95.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.600000,-0.800000,-4.060000


Olha que interessante, a função agg permite que você crie sua própria tabela descritiva. Ao invés do describe(), você pode criar algo com o agg e as estatísticas que mais atendem às suas necessidades, incluindo ainda funções que você mesmo tenha criado:

In [13]:
# Função de amplitude
def amplitude(series):
    return series.max() - series.min()

# Selecionar apenas as colunas numéricas
numeric_cols = stud.select_dtypes(include=[np.number])

# Aplicar funções de agregação apenas às colunas numéricas
numeric_cols.agg(['sum', 'mean', 'min', 'max', amplitude])



,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
sum,5214.000000,82592.000000,7644.000000,3.918179e+07,3941.000000,20252.000000,586681.300000,8287.000000,86542.000000,98546.000000,...,609.000000,2397.000000,27571.000000,35672.000000,19624.000000,45258.430117,665.000000,51168.600000,5432.800000,8.710000
mean,1.178571,18.669078,1.727848,8.856643e+03,0.890823,4.577758,132.613314,1.873192,19.561935,22.275316,...,0.137658,0.541817,6.232143,8.063291,4.435805,10.230206,0.150316,11.566139,1.228029,0.001969
min,1.000000,1.000000,0.000000,3.300000e+01,0.000000,1.000000,95.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.600000,-0.800000,-4.060000
max,6.000000,57.000000,9.000000,9.991000e+03,1.000000,43.000000,190.000000,109.000000,44.000000,44.000000,...,12.000000,19.000000,23.000000,33.000000,20.000000,18.571429,12.000000,16.200000,3.700000,3.510000
amplitude,5.000000,56.000000,9.000000,9.958000e+03,1.000000,42.000000,95.000000,108.000000,43.000000,43.000000,...,12.000000,19.000000,23.000000,33.000000,20.000000,18.571429,12.000000,8.600000,4.500000,7.570000


# 3. Unindo Diferentes Datasets

Uma das tarefas mais comuns no mercado de trabalho é você precisar unir diferentes datasets. Pense comigo, a empresa não consegue guardar todas as informações em um único local, não é sequer a melhor solução, a mais otimizada. A maioria das empresas vai ter um dataset de cadastro dos clientes, um contendo as compras feitas, um com informações de geolocalização e por aí vai. Agora, vamos pensar num exemplo real que acontece a todo o tempo nas empresas: imagine que o seu chefe peça para você levantar a informação de qual estado possui os maiores consumidores, em termos de dinheiro gasto, da sua empresa . Basicamente, você vai precisar pegar a informação de geolocalização e unir com a informação de compras. O que você precisará fazer: primeiro, resumir as informações de compras, agrupando e somando o que cada cliente gastou. Na sequência, você vai "plugar" a informação de geolocalização na informação de compras feitas. Por fim, precisaremos fazer um novo agrupamento, dessa vez, por estado. Abaixo, temos um desenho do processo que precisaria ser feito:

In [14]:
# Criando o dataframe "compras"
compras = pd.DataFrame({
'id_cliente': ['AA00', 'AA00', 'BB01', 'BB01', 'BB01', 'CC02', 'CC02', 'CC02', 'CC02', 'CC02'],
'data_compra': ['2023-01-01', '2023-01-05', '2023-01-10', '2023-01-15', '2023-01-20',
'2023-01-25', '2023-01-30', '2023-02-01', '2023-02-05', '2023-02-10'],
'valor_compra': [100, 150, 200, 250, 300, 350, 400, 450, 500, 550]
})

# Criando o dataframe "geolocalizacao"
geolocalizacao = pd.DataFrame({
'id_cliente': ['AA00', 'BB01', 'CC02', 'DD03', 'EE04'],
'estado': ['São Paulo', 'São Paulo', 'Minas Gerais', 'Bahia', 'Pernambuco']
})

Se quiser, pode dar uma conferida nas tabelas criadas. Nem precisa usar .head(), pois elas são pequenas, justamente para conseguirmos acompanhar as transformações:

In [15]:
compras

,id_cliente,data_compra,valor_compra
0,AA00,2023-01-01,100
1,AA00,2023-01-05,150
2,BB01,2023-01-10,200
3,BB01,2023-01-15,250
4,BB01,2023-01-20,300
5,CC02,2023-01-25,350
6,CC02,2023-01-30,400
7,CC02,2023-02-01,450
8,CC02,2023-02-05,500
9,CC02,2023-02-10,550


In [16]:
geolocalizacao

,id_cliente,estado
0,AA00,São Paulo
1,BB01,São Paulo
2,CC02,Minas Gerais
3,DD03,Bahia
4,EE04,Pernambuco


No dia a dia da empresa, se você estiver inseguro quanto ao que está fazendo, recomendo muito fazer algo parecido. Crie uma tabela temporária, contendo apenas 5 linhas da tabela que irá receber as informações (no nosso caso, seria "compras") e vá acompanhando se o que está sendo feito bate com o resultado esperado. Depois, execute o código com todas as linhas! Isso vai garantir que o que está sendo feito está correto e ainda pode te poupar tempo, já que ir executando tudo com a tabela inteira, errando e fazendo ajustes, pode demorar muito mais!

Bom, agora, vamos sumarizar a tabela compras. Precisamos somar as compras por cada id_cliente:

In [17]:
compras_group = compras.groupby('id_cliente')['valor_compra'].sum().reset_index()
compras_group

,id_cliente,valor_compra
0,AA00,250
1,BB01,750
2,CC02,2250


O groupby nós aprendemos semana passada, a única "inovação" de hoje é que o reset_index reseta o índice e mantém o formato de DataFrame para continuarmos trabalhando em cima destes dados. Agora, vem a parte que esta seção é dedicada: o join/merge! É hora de trazer informações de um outro DataFrame para o que estamos utilizando. Vamos fazer isso com um merge, na coluna id_cliente, do tipo left - ou seja, fixamos o DataFrame da esquerda, o inicial, e "plugamos" a informação do DataFrame à direita:

In [18]:
compras_group_geo = compras_group.merge(geolocalizacao, on='id_cliente',how='left')
compras_group_geo

,id_cliente,valor_compra,estado
0,AA00,250,São Paulo
1,BB01,750,São Paulo
2,CC02,2250,Minas Gerais


In [19]:
compras_group_geo = compras_group_geo.groupby('estado')['valor_compra'].sum().reset_index()
compras_group_geo

,estado,valor_compra
0,Minas Gerais,2250
1,São Paulo,1000


# Função merge

* É uma função no Pandas que fornece uma interface mais genérica para combinar DataFrames com base em colunas ou índices comuns.
* É mais flexível e permite especificar como deseja combinar os DataFrames (por exemplo, left, right, outer, inner).
* Sintaxe: pd.merge(df1, df2, how='inner', on='coluna_chave')
* É geralmente usada quando você deseja combinar DataFrames com base em colunas específicas.

# Função join

* É um método de um DataFrame que é uma abreviação conveniente para combinar DataFrames com base em seus índices. Por padrão, ele combina usando    * índices, mas você também pode especificar uma coluna.
* Também permite especificar o tipo de junção, mas é mais comumente usado para junções baseadas em índices.
* Sintaxe: df1.join(df2, on='coluna_chave')
* É mais conveniente de usar quando você deseja combinar DataFrames com base em seus índices.

Agora que você já entende, vamos falar um pouco do merge que fizemos. A sintaxe é bem simples, você passa os 2 dataframes que quer unir, sendo o primeiro o DataFrame que receberá informação e o segundo é de onde vamos trazer a informação. Agora, existem diversas formas de trazer a informação. Veja que, no nosso caso, nós fixamos o DataFrame compras e trouxemos a informação de geolocalização. Se a gente fixasse o de geolocalização, o resultado seria diferente, já que temos ids nele e que não estão no DataFrame de compras. Poderíamos também ter o caso de ter ids no primeiro DataFrame que não estão no segundo e precisaríamos decidir se vamos só agregar informação do segundo DataFrame ou se iríamos apenas trazer a intersecção entre os 2. Enfim, são várias formas diferentes e é o parâmetro how que define como os DataFrames serão unidos. Vejamos uma ilustração para ajudar a como decidir se você quer uma função merge com inner, left, right, etc:

![teste](https://harmash.com/tutorials/sql/self-join/eca2f76f-1266-4eb1-b654-a97922a676dc_type-of-joins.PNG)

1. Left (Junção à Esquerda):
- A junção à esquerda usa apenas as chaves do DataFrame à esquerda.
- O resultado conterá todas as linhas do DataFrame à esquerda e as linhas correspondentes do DataFrame à direita. Se não houver correspondência para uma linha do DataFrame à esquerda, os valores nas colunas do DataFrame à direita serão 'NaN'.

2. Right (Junção à Direita):
- A junção à direita usa apenas as chaves do DataFrame à direita.
- O resultado conterá todas as linhas do DataFrame à direita e as linhas correspondentes do DataFrame à esquerda. Se não houver correspondência para uma linha do DataFrame à direita, os valores nas colunas do DataFrame à esquerda serão 'NaN'.

3. Inner (Junção Interna):
- A junção interna usa apenas as chaves comuns aos dois DataFrames.
- O resultado conterá apenas as linhas para as quais há chaves correspondentes em ambos os DataFrames. Linhas em ambos os DataFrames que não têm correspondências serão excluídas.

4. Outer (Junção Externa ou Full Outer Join):
- A junção externa usa todas as chaves presentes em ambos os DataFrames.
- O resultado conterá todas as linhas de ambos os DataFrames. Para as linhas que não têm correspondências no outro DataFrame, os valores nas colunas desse DataFrame serão 'NaN'.

In [20]:
df_A = pd.DataFrame({
    'key': [1, 2, 3],
    'Value_A': ['A1', 'A2', 'A3']
})

df_B = pd.DataFrame({
    'key': [1, 3, 4],
    'Value_B': ['B1', 'B3', 'B4']
})

# Exibindo os DataFrames para verificar


In [21]:
df_A

,key,Value_A
0,1,A1
1,2,A2
2,3,A3


In [22]:
df_B

,key,Value_B
0,1,B1
1,3,B3
2,4,B4


In [23]:
left_join = df_A.merge(df_B,on='key', how='left')
print('Left Join:')
left_join

Left Join:


,key,Value_A,Value_B
0,1,A1,B1
1,2,A2,NaN
2,3,A3,B3


In [24]:
right_join = df_A.merge(df_B, on='key', how='right')
print('Right Join:')
right_join

Right Join:


,key,Value_A,Value_B
0,1,A1,B1
1,3,A3,B3
2,4,NaN,B4


In [25]:
inner_join = df_A.merge(df_B, on='key', how='inner')
print('Inner Join')
inner_join

Inner Join


,key,Value_A,Value_B
0,1,A1,B1
1,3,A3,B3


In [26]:
outer_join = df_A.merge(df_B, on='key', how='outer')
print("\nOuter Join:")
outer_join


Outer Join:


,key,Value_A,Value_B
0,1,A1,B1
1,2,A2,NaN
2,3,A3,B3
3,4,NaN,B4


Agora, algo menos comum, mas que pode ocorrer, é a necessidade de empilharmos os DataFrames. Imagine que a gente tenha as informações das compras dos clientes de Junho e Julho em um DataFrame e as compras de Agosto e Setembro em outro DataFrame. Como podemos fazer essa união? É nesta que entra a função concat:

In [27]:
# Criando o DataFrame de compras de junho e julho
jun_jul = pd.DataFrame({
'id_cliente': ['AA01', 'BB02', 'AA01', 'CC03', 'BB02', 'DD04'],
'dt_compra': ['2023-06-05', '2023-06-15', '2023-06-25', '2023-07-05', '2023-07-15', '2023-07-25'],
'vl_compra': [200.50, 155.75, 333.60, 450.00, 300.10, 250.00]
})


# Criando o DataFrame de compras de agosto e setembro
ago_set = pd.DataFrame({
'id_cliente': ['EE05', 'EE05', 'FF06', 'GG07'],
'dt_compra': ['2023-08-05', '2023-08-15', '2023-08-25', '2023-09-05'],
'vl_compra': [205.55, 233.75, 550.65, 320.82]
})

In [28]:
jun_jul

,id_cliente,dt_compra,vl_compra
0,AA01,2023-06-05,200.50
1,BB02,2023-06-15,155.75
2,AA01,2023-06-25,333.60
3,CC03,2023-07-05,450.00
4,BB02,2023-07-15,300.10
5,DD04,2023-07-25,250.00


In [29]:
ago_set

,id_cliente,dt_compra,vl_compra
0,EE05,2023-08-05,205.55
1,EE05,2023-08-15,233.75
2,FF06,2023-08-25,550.65
3,GG07,2023-09-05,320.82


In [30]:
df_concat = pd.concat([jun_jul,ago_set])
df_concat

,id_cliente,dt_compra,vl_compra
0,AA01,2023-06-05,200.50
1,BB02,2023-06-15,155.75
2,AA01,2023-06-25,333.60
3,CC03,2023-07-05,450.00
4,BB02,2023-07-15,300.10
5,DD04,2023-07-25,250.00
0,EE05,2023-08-05,205.55
1,EE05,2023-08-15,233.75
2,FF06,2023-08-25,550.65
3,GG07,2023-09-05,320.82


# 4. Outros Métodos e Funções
# 4.1 Criando intervalos discretos com cut e qcut

O método cut é uma ferramenta poderosa do Pandas que permite segmentar e classificar os valores dos dados em compartimentos (ou bins) discretos. Isso é útil quando você deseja categorizar dados contínuos em intervalos específicos.

In [31]:
df = pd.DataFrame({'value': [1,2,3,4,5]})
df['bin'] = pd.cut(df['value'], bins=[0,3,5], labels=['Low','High'])
df

,value,bin
0,1,Low
1,2,Low
2,3,Low
3,4,High
4,5,High


In [32]:
df['equal_bins'] = pd.cut(df['value'], bins=3)
df

,value,bin,equal_bins
0,1,Low,"(0.996, 2.333]"
1,2,Low,"(0.996, 2.333]"
2,3,Low,"(2.333, 3.667]"
3,4,High,"(3.667, 5.0]"
4,5,High,"(3.667, 5.0]"


# 4.2 Método sample
O método sample é uma maneira conveniente de obter uma amostra aleatória de itens de um eixo de um DataFrame, seja por número ou por fração. Como muitos de nossos estudos são feitos por amostragem, ou também acontece de utilizarmos só parte da base durante a criação dos passos de um projeto grande, a amostragem acaba sendo muito útil!

In [33]:
# Amostra de 10 estudantes
amostra = stud.sample(n=10)
amostra

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
2112,1,1,2,9556,1,1,136.0,1,19,19,...,0,8,8,8,16.800000,0,12.7,3.7,-1.70,Graduate
1825,1,17,1,9254,1,1,160.0,1,19,37,...,0,6,7,6,14.166667,0,12.7,3.7,-1.70,Graduate
3020,1,17,1,9130,1,1,131.0,1,19,9,...,0,5,7,5,12.833333,0,7.6,2.6,0.32,Graduate
1753,1,44,1,9119,1,39,150.0,1,19,3,...,0,5,12,2,10.500000,0,12.7,3.7,-1.70,Enrolled
898,1,1,1,9254,1,1,112.0,1,38,19,...,0,6,10,4,11.000000,0,7.6,2.6,0.32,Enrolled
1593,1,1,3,171,1,1,129.0,1,38,37,...,0,0,0,0,0.000000,0,9.4,-0.8,-3.12,Dropout
489,1,1,3,9500,1,1,128.0,1,38,38,...,0,8,8,7,13.542857,0,13.9,-0.3,0.79,Graduate
3486,1,1,2,9500,1,1,149.0,1,19,1,...,0,8,8,7,13.000000,0,12.7,3.7,-1.70,Graduate
1234,1,17,6,9500,1,1,150.0,1,1,37,...,0,8,8,7,13.925714,0,10.8,1.4,1.74,Graduate
3301,1,17,1,9500,1,1,130.0,1,19,1,...,0,8,8,8,14.362500,0,11.1,0.6,2.02,Graduate


In [34]:
# Se você preferir, pode extrair um percentual do DataFrame:
amostra_10_perc = stud.sample(frac=0.1)
amostra_10_perc

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
342,2,39,1,8014,0,1,120.0,1,37,37,...,0,6,8,6,12.857143,0,12.4,0.5,1.79,Graduate
2506,1,1,1,9556,1,1,123.0,1,19,19,...,0,8,10,8,11.425000,0,12.7,3.7,-1.70,Graduate
786,1,1,1,9070,1,1,153.0,1,3,3,...,0,6,6,5,14.000000,0,12.7,3.7,-1.70,Graduate
2365,1,18,1,9773,1,1,125.0,1,1,1,...,0,6,6,6,14.500000,0,8.9,1.4,3.51,Graduate
127,1,1,1,9853,1,1,146.0,1,19,38,...,0,6,7,5,13.400000,0,9.4,-0.8,-3.12,Enrolled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,1,1,2,9147,1,1,128.0,1,1,1,...,0,5,13,2,13.500000,0,8.9,1.4,3.51,Enrolled
4184,1,1,1,9238,1,1,136.0,1,38,38,...,0,6,13,5,11.400000,0,7.6,2.6,0.32,Graduate
2123,1,17,3,9238,1,1,118.0,1,1,19,...,0,6,7,0,0.000000,0,16.2,0.3,-0.92,Dropout
1722,1,51,1,9085,1,1,127.0,1,3,3,...,5,12,20,12,15.428571,0,8.9,1.4,3.51,Graduate


Por padrão, a amostragem é sem reposição. Se quiser uma amostragem com reposição, utilize o parâmetro replace = True:

# 4.3 Método nunique e unique
A função nunique retorna o número de elementos únicos no objeto, enquanto unique retorna os elementos únicos em si.

In [35]:
# Retorna os valores únicos 
stud.Target.unique()

array(['Dropout', 'Graduate', 'Enrolled'], dtype=object)

In [36]:
stud.Target.nunique()

3

Quando se tem 100, 200, ..., 1000 colunas, fica difícil checar cada uma para separar entre numéricas em categóricas. Sendo assim, você terá duas opções para fazer essa separação: (1) colocar tudo que é texto como feature categórica e o restante como numérica; (2) Olhar quantos valores únicos cada coluna possui e definir um valor como o limite que separa as features numéricas de categóricas (ex.: abaixo de 10 valores únicos são categóricas e o restante são numéricas). Para o segundo caso, df.unique() vai te ajudar a fazer o mapeamento.

In [37]:
# Lista de colunas com tipo de dado 'object' (geralmente texto ou string)
categorical_features = stud.select_dtypes(include=['object']).columns.tolist()
# Lista de colunas com tipos de dados numéricos (int e float)
numerical_features = stud.select_dtypes(exclude=['object']).columns.tolist()


# 4.4 Como usar funções com apply

O método apply() é uma das ferramentas mais versáteis e poderosas do Pandas. Ele permite que você aplique uma função personalizada a cada elemento de uma Series ou a cada coluna/linha de um DataFrame. Isso é especialmente útil quando você precisa realizar transformações ou cálculos que não são facilmente alcançados com os métodos padrão do Pandas.

Para os exercícios de Pandas, vamos usar o data
Se você deseja aplicar uma função a cada elemento de uma Series, simplesmente use apply() diretamente nessa Series. Por exemplo, podemos calcular o quadrado de cada valor em uma coluna usando o apply combinado com uma função lambda:

In [38]:
df =pd.DataFrame({'A':[1,2,3,4,5]})

df['quadrado'] = df['A'].apply(lambda x: x**2)

df

,A,quadrado
0,1,1
1,2,4
2,3,9
3,4,16
4,5,25


Voltando ao nosso dataset de alunos, vamos criar uma nova coluna para deixar o GDP multiplicado por 100:

In [39]:
stud['new_GDP'] = stud.GDP.apply(lambda x: x*100)
stud[['GDP','new_GDP']].head()

,GDP,new_GDP
0,1.74,174.0
1,0.79,79.0
2,1.74,174.0
3,-3.12,-312.0
4,0.79,79.0


Ao usar apply() em um DataFrame, você pode especificar o eixo ao longo do qual a função deve ser aplicada. Por padrão, a função será aplicada a cada coluna (axis=0), mas você pode alterar para aplicar a função a cada linha (axis=1). Veja como podemos aplicar nas colunas e linhas de um DataFrame:

In [40]:
# Construindo dataframe para exemplos
df = pd.DataFrame({
    'A': [1,2,3],
    'B': [4,5,6],
    'C': [7,8,9]
})

df

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [41]:
# Media das colunas
df.apply(lambda col: col.mean(), axis=1)

0    4.0
1    5.0
2    6.0
dtype: float64

In [42]:
# Soma das linhas
df.apply(lambda row: row.sum(), axis=0)

A     6
B    15
C    24
dtype: int64

O método apply() é extremamente flexível e, com ele, as possibilidades são quase infinitas. Seja para transformações simples ou operações mais complexas, apply() é uma ferramenta essencial no arsenal de qualquer cientista ou analista de dados que trabalhe com Pandas

# 4.5 Tabelas de Frequência Cruzada

A função crosstab() é uma ferramenta poderosa do Pandas que permite criar tabelas de contingência, que são ótimas para entender a relação entre duas variáveis categóricas.

Vamos ao nosso dataset de bikes! Baixe no anexo o arquivo bike.zip, vamos carregar o dataset products:

In [43]:
pd.crosstab(prod['model_year'], prod['brand_id'])

brand_id,1,2,3,4,5,6,7,8,9
model_year,,,,,,,,,
2016,12,0,1,3,1,0,0,4,5
2017,16,10,0,0,0,0,23,8,28
2018,90,0,2,0,0,3,0,13,96
2019,0,0,0,0,0,0,0,0,6


Veja que apenas a marca de id 9 criou bikes no ano de 2019. Além disso, podemos ver que ela é uma das que mais criou produtos, junto com a marca 1.

Se quisermos ver o total das linhas e colunas:

In [44]:
pd.crosstab(prod['model_year'], prod['brand_id'],margins = True)

brand_id,1,2,3,4,5,6,7,8,9,All
model_year,,,,,,,,,,
2016,12,0,1,3,1,0,0,4,5,26
2017,16,10,0,0,0,0,23,8,28,85
2018,90,0,2,0,0,3,0,13,96,204
2019,0,0,0,0,0,0,0,0,6,6
All,118,10,3,3,1,3,23,25,135,321


Agora ficou mais claro que a marca 9 é a que mais produz e o ano de 2018 foi o ano que mais se criou produtos para este grupo de marcas.

Se quisermos ver quanto cada célula representa daquela linha ou daquela coluna, em percentual, aí teremos que ser um pouco mais espertos e nos aproveitarmos do apply:

In [45]:
# percentual da linha
pd.crosstab(prod['model_year'], prod['brand_id']).apply(lambda r: r/r.sum(),axis=1)

brand_id,1,2,3,4,5,6,7,8,9
model_year,,,,,,,,,
2016,0.461538,0.000000,0.038462,0.115385,0.038462,0.000000,0.000000,0.153846,0.192308
2017,0.188235,0.117647,0.000000,0.000000,0.000000,0.000000,0.270588,0.094118,0.329412
2018,0.441176,0.000000,0.009804,0.000000,0.000000,0.014706,0.000000,0.063725,0.470588
2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [46]:
# Calcular o percentual da linha * 100
crosstab = pd.crosstab(prod['model_year'], prod['brand_id']).apply(lambda r: (r/r.sum())*100, axis=1)

# Formatar os valores como porcentagem com duas casas decimais e incluir o símbolo %
crosstab.style.format("{:.2f}%")

brand_id,1,2,3,4,5,6,7,8,9
model_year,,,,,,,,,
2016,46.15%,0.00%,3.85%,11.54%,3.85%,0.00%,0.00%,15.38%,19.23%
2017,18.82%,11.76%,0.00%,0.00%,0.00%,0.00%,27.06%,9.41%,32.94%
2018,44.12%,0.00%,0.98%,0.00%,0.00%,1.47%,0.00%,6.37%,47.06%
2019,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%


In [47]:
# Percentual da coluna * 100
crosstab_coluna = pd.crosstab(prod['model_year'], prod['brand_id']).apply(lambda r: (r/r.sum())*100, axis=0)

crosstab_coluna


brand_id,1,2,3,4,5,6,7,8,9
model_year,,,,,,,,,
2016,10.169492,0.0,33.333333,100.0,100.0,0.0,0.0,16.0,3.703704
2017,13.559322,100.0,0.000000,0.0,0.0,0.0,100.0,32.0,20.740741
2018,76.271186,0.0,66.666667,0.0,0.0,100.0,0.0,52.0,71.111111
2019,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,4.444444


In [48]:
# Função para destacar máximo, mínimo e intermediários
def highlight_max_min(data):
    styles = pd.DataFrame('', index=data.index, columns=data.columns)
    for col in data.columns:
        max_val = data[col].max()
        min_val = data[col].min()
        styles[col] = ['background-color: lightgreen' if v == max_val else 'background-color: blank'
                       if v == min_val else 'background-color: orange' for v in data[col]]
    return styles

# Aplicar formatação e estilização
styled_df = crosstab_coluna.style.apply(highlight_max_min, axis=None).format("{:.2f}%")

# Exibir o DataFrame estilizado
styled_df

brand_id,1,2,3,4,5,6,7,8,9
model_year,,,,,,,,,
2016,10.17%,0.00%,33.33%,100.00%,100.00%,0.00%,0.00%,16.00%,3.70%
2017,13.56%,100.00%,0.00%,0.00%,0.00%,0.00%,100.00%,32.00%,20.74%
2018,76.27%,0.00%,66.67%,0.00%,0.00%,100.00%,0.00%,52.00%,71.11%
2019,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,4.44%


# 4.6 Identificando Valores Ausentes com isna()
A função isna() é usada para identificar valores ausentes em um DataFrame ou Series. Ele retorna uma máscara booleana (Verdadeiro ou Falso) do mesmo tamanho que os dados, onde True indica um valor ausente.
Exemplo básico:

In [49]:
import pandas as pd

# Corrigindo as listas para que tenham o mesmo comprimento
df = pd.DataFrame({
    'A': [1, 2, None, 4, 5],
    'B': ['A', None, 'C', 'D', 'E']
})

df


,A,B
0,1.0,A
1,2.0,None
2,NaN,C
3,4.0,D
4,5.0,E


In [50]:
df.isna()

,A,B
0,False,False
1,False,True
2,True,False
3,False,False
4,False,False


Contando valores ausentes por coluna:


In [51]:
# ausentes por coluna (soma)
df.isna().sum()

A    1
B    1
dtype: int64

In [52]:
# ausentes por coluna (percentual)
df.isna().mean()

A    0.2
B    0.2
dtype: float64

Filtrando linhas com valores ausentes:

In [53]:
# Filtrando missing na coluna A
df[df['A'].isna()]

,A,B
2,NaN,C


In [54]:
# Filtrando missing na coluna B
df[df['B'].isna()]

,A,B
1,2.0,None


# 4.7 Filtrando Dados com query()
O método query() permite filtrar dados de um DataFrame usando uma string de consulta, o que pode ser mais legível e conciso do que a indexação booleana tradicional. Em outras palavras, o que ele recebe é uma lógica que resulta em Verdadeiro ou Falso e ele retorna quando aquilo for verdadeiro. Voltando ao dataset products do nosso arquivo bike.zip, veja como filtramos facilmente todas as bikes cujo preço é superior a 5500:

In [55]:
prod.query('list_price > 5500')

,product_id,product_name,brand_id,category_id,model_year,list_price
49,50,Trek Silque SLR 7 Women's - 2017,9,7,2017,5999.99
50,51,Trek Silque SLR 8 Women's - 2017,9,7,2017,6499.99
148,149,Trek Domane SLR 8 Disc - 2018,9,7,2018,7499.99
154,155,Trek Domane SLR 9 Disc - 2018,9,7,2018,11999.99
155,156,Trek Domane SL Frameset - 2018,9,7,2018,6499.99
156,157,Trek Domane SL Frameset Women's - 2018,9,7,2018,6499.99
168,169,Trek Emonda SLR 8 - 2018,9,7,2018,6499.99


Veja que o que está entre parênteses é uma lógica que só poderia retornar True ou False. O que query() faz é nos mostrar quais linhas correspondem à condição quando ela for verdadeira.

É possível utilizar operadores lógicos dentro de query():

In [56]:
prod.query('list_price > 5500 & model_year == 2018')

,product_id,product_name,brand_id,category_id,model_year,list_price
148,149,Trek Domane SLR 8 Disc - 2018,9,7,2018,7499.99
154,155,Trek Domane SLR 9 Disc - 2018,9,7,2018,11999.99
155,156,Trek Domane SL Frameset - 2018,9,7,2018,6499.99
156,157,Trek Domane SL Frameset Women's - 2018,9,7,2018,6499.99
168,169,Trek Emonda SLR 8 - 2018,9,7,2018,6499.99


Podemos também referenciar valores externos:

In [57]:
preco = 550
prod.query('list_price > @preco & model_year == 2018')

,product_id,product_name,brand_id,category_id,model_year,list_price
113,114,Trek Marlin 6 - 2018,9,6,2018,579.99
114,115,Trek Fuel EX 8 29 - 2018,9,6,2018,3199.99
115,116,Trek Marlin 7 - 2017/2018,9,6,2018,749.99
116,117,Trek Ticket S Frame - 2018,9,6,2018,1469.99
117,118,Trek X-Caliber 8 - 2018,9,6,2018,999.99
...,...,...,...,...,...,...
309,310,Electra Townie Commute 27D Ladies - 2018,1,2,2018,899.99
310,311,Electra Townie Commute 8D - 2018,1,2,2018,749.99
311,312,Electra Townie Commute 8D Ladies' - 2018,1,2,2018,699.99
313,314,Electra Townie Original 21D EQ Ladies' - 2018,1,2,2018,679.99


Com isso, creio que finalizamos a parte teórica desta semana. Não conseguimos ver tudo que há no Pandas, acho que nunca conseguiremos usar todas as inúmeras funcionalidades que ele possui, mas já vimos o bastante para fazermos análises de peso! No fim deste material, deixei uma série de exercícios para testar seus conhecimentos! Vamos ver o quão preparados estamos para os dados reais, fechou?


# 5. Considerações Finais

Neste material, mergulhamos profundamente em funções avançadas que enriquecem nossa habilidade de manipulação e análise de dados. Abordamos desde a construção de tabelas dinâmicas, uma técnica amplamente reconhecida no Excel, até outras funcionalidades como a categorização de dados e a identificação de valores únicos. Cada técnica e ferramenta discutida aqui é uma alavanca poderosa para aprimorar sua eficiência analítica e te tornar um profissional de alto nível no mercado profissional - bora se destacar aí!

Além disso, destacamos práticas indispensáveis no mundo corporativo: as junções, fundamentais devido à estruturação dos bancos de dados, e a formatação condicional, que confere sofisticação e clareza aos relatórios. Esta jornada é projetada para equipá-lo com habilidades essenciais e insights valiosos para suas análises.

À medida que você avança em sua carreira de ciência de dados, descobrirá que muitas vezes a preparação e a manipulação de dados são etapas que consomem mais tempo do que a modelagem em si. Portanto, dominar as habilidades apresentadas neste material é crucial. Agora, para solidificar seu aprendizado e garantir que você esteja pronto para aplicar essas técnicas no mundo real, encorajamos você a mergulhar na seção de Exercícios. Nela, você encontrará desafios práticos que testarão e aprimorarão sua compreensão. Boa sorte e bons estudos!



# 6. Exercícios
Utilize o arquivo bank.zip para responder às perguntas:

1) Olhando apenas para o grupo de divorciados, qual o percentual deles que é empreendedor (entrepreneur)? E estudantes?

In [58]:
bnk = pd.read_csv('/kaggle/input/bank-marketing/bank-full.csv', sep=";")
bnk.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [59]:
# Filtrando todos os dados para incluir apenas os divorciados

divorciados_df = bnk[bnk['marital'] == 'divorced']

# Calculando o total de individuos divorciados
total_divorciados = len(divorciados_df)

# Agrupar por job e calcular a contagem
job_counts = divorciados_df['job'].value_counts()

# Calcular o percentual de empreendedores e estudantes
percent_empreendedores = (job_counts.get('entrepreneur', 0) / total_divorciados) * 100
percent_estudantes = (job_counts.get('student', 0) / total_divorciados) * 100

print(f"percentual de empreendedores entre divorciados: {percent_empreendedores:.2f}%")
print(f"percentual de estudantes entre divorciados: {percent_estudantes:.2f}%")

percentual de empreendedores entre divorciados: 3.44%
percentual de estudantes entre divorciados: 0.12%


2) O seu chefe decidiu criar duas categorias, Premium e Gold, para focar nos clientes mais promissores. Para a categoria Premium, entrarão os que possuem saldo (balance) nos top 10%. Ou seja, o grupo acima do percentil 90. Já o grupo Gold, ficarão os que possuem saldo nos top 20%, mas abaixo do grupo Premium. Ou seja, do p80 até o último do p90. Verifique quantos clientes possuem no grupo Premium e no Gold.

In [60]:
# Calcular os percentis 90 e 80 para balance

percentil_90 = bnk['balance'].quantile(0.90)
percentil_80 = bnk['balance'].quantile(0.80)

# Filtrar os clientes do grupo Premium (acima do percentil 90)
premium_group = bnk[bnk['balance'] > percentil_90]

# Filtrar os clientes do grupo Gold (entre os percentis 80 e 90)
gold_group = bnk[(bnk['balance']< percentil_90)& (bnk['balance'] >= percentil_80)]

# Contar o número de clientes em cada grupo
num_premium = premium_group.shape[0]
num_gold = gold_group.shape[0]

print(f"Número de clientes no grupo Premium: {num_premium}")
print(f"Número de clientes no grupo Premium: {num_gold}")

Número de clientes no grupo Premium: 4521
Número de clientes no grupo Premium: 4525


3) Seu chefe também quer saber se existe diferença nos percentuais vistos no exercício (1) para o grupo Premium. Verifique como a tabela cruzada da população se diferencia do grupo Premium. Existe diferença? Qual sua hipótese para que exista essa diferença?

In [61]:
# Filtrar clientes Premium
percentil_90 = bnk['balance'].quantile(0.90)
premium_customers = bnk[bnk['balance'] > percentil_90]

# Filtrar divorciados no grupo Premium
divorced_premium = premium_customers[premium_customers['marital'] == 'divorced']

# Criar a tabela cruzada para divorciados no grupo Premium
crosstab_divorced_premium = pd.crosstab(divorced_premium['marital'], divorced_premium['job'], normalize='index') * 100

# Formatando os valores como strings com duas casas decimais e o símbolo de porcentagem
crosstab_divorced_premium = crosstab_divorced_premium.apply(lambda x: x.map(lambda v: f"{v:.2f}%"))

crosstab_divorced_premium

job,admin.,blue-collar,entrepreneur,housemaid,management,retired,self-employed,services,technician,unemployed,unknown
marital,,,,,,,,,,,
divorced,9.61%,9.61%,2.75%,5.26%,30.66%,12.36%,5.72%,7.78%,11.67%,3.89%,0.69%


5) Na sua opinião e de parte da diretoria, há 3 fatores que determinam se a pessoa vai aceitar a proposta do seu banco (y): o saldo (balance), se ela já deixou de pagar um empréstimo (default) e se ela tem um empréstimo pessoal (loan). Para avaliar essas 3 características junto com o aceite (y), você precisará criar uma tabela dinâmica, cujo valor observado seja a média do saldo. Avalie se existe diferença no saldo de quem aceita a proposta e tem empréstimo vs quem não aceita e não tem empréstimo, e o mesmo para o default. Em outras palavras, crie uma tabela dinâmica cruzando loan e default, mas também segmentando por aceite ou não da proposta. O saldo de quem aceita e tem empréstimo é maior ou menor do que o de quem não aceita? Por qual motivo você acha que isso acontece? Crie a sua hipótese para justificar o resultado achado!

In [62]:
bnk_renamed = bnk.rename(columns={
    'y': 'proposta',
    'default': 'caloteiro',
    'loan': 'tem_emprestimo',
    'balance': 'saldo'
})

# Criar a tabela dinâmica com os novos nomes
bnk_renamed.pivot_table(
    index=['proposta', 'caloteiro'],
    columns='tem_emprestimo',
    values='saldo',
    aggfunc='mean'
)


tem_emprestimo               no         yes
proposta caloteiro                         
no       no         1435.673581  807.983318
         yes        -124.737945 -172.958042
yes      no         1912.359060  912.245203
         yes         -82.108108  -10.666667

O saldo de quem aceita uma proposta tende a ser maior de quem não aceita a proposta por conta dos juros que acabam sendo descontados a um valor bem menor quando é feita uma negociação com o banco, e também vemos que o menor saldo é o do quadrante onde não é aceita proposta e o individuo é caloteiro

7) Você trabalha numa empresa de bicicletas e precisa fazer uma clusterização separando marcas caras de baratas. Para isso, você precisará saber a média de preços das bicicletas de cada marca.

Dica: No dataset brands, você encontra as marcas e no products você possui o preço de cada bicicleta. Traga a informação de preço de products para brands usando a coluna brand_id, e, em seguida, faça o agrupamento de preço médio por cada brand.

In [63]:
import pandas as pd

# Carregando os datasets
brands = pd.read_csv('/kaggle/input/bike-store-sample-database/brands.csv')
products = pd.read_csv('/kaggle/input/bike-store-sample-database/products.csv')

# Combinando os datasets na coluna 'brand_id'
merged_df = pd.merge(brands, products, on='brand_id')

# Agrupando pelo nome da marca e calculando a média dos preços
average_prices = merged_df.groupby('brand_name')['list_price'].mean().reset_index()

# Calculando o total geral
total_geral = average_prices['list_price'].sum()

# Criando a coluna de porcentagem formatada em relação ao total
average_prices['porcentagem'] = (average_prices['list_price'] / total_geral) * 100
average_prices['porcentagem'] = average_prices['porcentagem'].map(lambda x: f"{x:.2f}%")

# Adicionando a linha de total
total_row = pd.DataFrame({'brand_name': ['Total'], 'list_price': [total_geral], 'porcentagem': ['100.00%']})
average_prices = pd.concat([average_prices, total_row], ignore_index=True)

# Função para aplicar o gradiente de cor apenas para linhas não totais
def color_gradient(row):
    if row['brand_name'] == 'Total':
        return ['background-color: blank'] * len(row)
    normalized_val = (row['list_price'] - average_prices['list_price'].min()) / (average_prices['list_price'].max() - average_prices['list_price'].min())
    # Ajuste o gradiente com mais saturação e menos luminosidade
    color = f'hsl(0, {normalized_val * 100}%, {100 - (normalized_val * 50)}%)'
    return [f'background-color: {color}'] * len(row)

# Função para destacar a linha total
def highlight_total(row):
    if row['brand_name'] == 'Total':
        return ['background-color: lightgrey'] * len(row)
    return [''] * len(row)

# Aplicando os estilos usando Styler.apply
styled_average_prices = average_prices.style.apply(color_gradient, axis=1).apply(highlight_total, axis=1)

styled_average_prices


,brand_name,list_price,porcentagem
0,Electra,761.006186,8.17%
1,Haro,621.990000,6.68%
2,Heller,2172.996667,23.33%
3,Pure Cycles,442.333333,4.75%
4,Ritchey,749.990000,8.05%
5,Strider,209.990000,2.25%
6,Sun Bicycles,524.468261,5.63%
7,Surly,1331.753600,14.30%
8,Trek,2500.064074,26.84%
9,Total,9314.592121,100.00%
